# IRAF-like: photutils

[photutils]() provides a couple of options for stellar source detection that will be familiar to users of IRAF. One is DAOFIND and the other is IRAF's starfind. The recommendation is to use DAOFIND because it is more general than starfind (e.g. it allows elliptical sources) and detects more sources. This notebook will focus on DAOFIND, implemented in photutils by the class `DAOStarFinder`.

Both methods find sources above a threshold that is specified as a multiple of the background noise level, and both require that the background be subtracted from the image.

You can use any of the background subtraction methods that you like; often simply subtracting the median will be adequate, which is what we will do in this notebook. 

## DAOPHOT

DAOStarFinder is a class that provides an implementation of the DAOFIND algorithm (Stetson 1987, PASP 99, 191). It searches images for local density maxima that have a peak amplitude greater than a specified threshold (the threshold is applied to a convolved image) and have a size and shape similar to a defined 2D Gaussian kernel. DAOStarFinder also provides an estimate of the objects’ roundness and sharpness, whose lower and upper bounds can be specified.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from photutils.datasets import load_star_image
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture

from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.stats import sigma_clipped_stats, SigmaClip


In [ ]:
# photutils allows us to easily access one of their example images
hdu = load_star_image()  
# We will only consider a small portion of the image, for faster processing
data = hdu.data[0:401, 0:401]  

# Plot the image with a square-root normalization
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, norm=norm, origin='lower', cmap='Greys_r')

In [ ]:
# Use sigma-clipped statistics to estimate the background & background noise
clipped_mean, clipped_med, clipped_std = sigma_clipped_stats(data, sigma=3.0) 
print(clipped_mean, clipped_med, clipped_std)

Now we will subtract the background and use an instance of DAOStarFinder to find the stars in the image that have FWHMs of around 3 pixels and have peaks approximately 5-sigma above the background. Running this class on the data yields an astropy Table containing the results of the star finder.

Note that at this point, the star positions are given in pixel coordinates (x and y). The locations are not restricted by the pixel grid; DAOfind will estimate the centroid position to 1% of a pixel. 

In [ ]:
daofind = DAOStarFinder(fwhm=3.0, threshold=5.*clipped_std)  
sources = daofind(data - clipped_med)  
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  

Let’s plot the image and mark the location of detected sources

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
_ = apertures.plot(color='blue', lw=1.5, alpha=0.5)

## Your turn - change the arguments for DAOStarFinder and see how they affect the sources identified